In [1]:
# Imports

import time
from datetime import date

Diplomarbeit Kaffeevollautomat

Datenschema + Demodaten

Betriebsbereite Maschine

Maschine reinigt sich 0:02 - 0:57

0:22 - 0:27 Brühgruppe fährt nach oben
0:33 - 0:51 Wasser läuft durch die Maschine und kommt vorne wieder raus
0:51 - 0:56 Brühgruppe fährt wieder nach unten

Betriebsbereite Maschine

Kaffee rausdrücken 1:04 - 1:42

1:04 - Knopf wird gedrückt
1:04 - 1:10 Kaffee wird gemahlen
1:10 - 1:15 Brühgruppe fährt nach oben / Gemahlener Kaffee wird gepresst
1:18 - 1:20 Pulver wird kurz angefeuchtet und die Temperatur des Wassers wird reguliert
1:24 - 1:38 Kaffee wird gebrüht
1:38 - Kaffee fertig zur Entnahme
1:38 - 1:42 Brühgruppe fährt in Ausgangsposition

Demodaten Kaffee rausdrücken

In [2]:
class MachineState:
    def __init__(self, temp=22):
        self.temp = temp
        self.water_ok = True
        self.grounds_ok = True
        self.cups_since_empty = 0
        self.water_flow = 0
        self.date = date.today 

    def __repr__(self):
        water_status = "OK" if self.water_ok else "Leer"
        grounds_status = "OK" if self.grounds_ok else "Voll"
        return (f"Temp: {self.temp}°C, Wasser: {water_status}, "
                f"Kaffeesatz: {grounds_status}, Wasserdurchfluss: {self.water_flow}ml/s, Datum: {date.today().strftime("%d.%m.%Y")}")

class Step:
    def __init__(self, second: int, description: str):
        self.second = second
        self.description = description

    def execute(self, state: MachineState):
        print(f"{self.description} | {state}")
        time.sleep(1)

class HeatUp(Step):
    def __init__(self, second: int, temp: int):
        super().__init__(second, f"Aufheizen")
        self.target_temp = temp
    
    def execute(self, state: MachineState):
        state.temp = self.target_temp

        if 31 <= self.second <= 49:
            state.water_flow = 5
        else:
            state.water_flow = 0

        super().execute(state)

        if self.second >= 49 or self.second < 31:
            state.water_flow = 0

class CoffeeGrind(Step): pass
class CoffeePress(Step): pass
class PowderMoister(Step):
    def execute(self, state: MachineState):
        state.water_flow = 5  

class CoffeeBrewing(Step):
    def execute(self, state: MachineState):
        if not state.water_ok:
            print(">>> Nicht genug Wasser für den Kaffee! Bitte Wasser nachfüllen.")
            return False
        if not state.grounds_ok:
            print(">>> Kaffeesatzbehälter voll! Bitte leeren.")
            return False

        state.water_flow = 5  
        
        if state.cups_since_empty >= 22:
            state.grounds_ok = False

        print(f"{self.description} | {state}")
        time.sleep(1)
        state.water_flow = 0  
        return True

class BrewingUnitToStartPosition(Step): pass

grinding = [CoffeeGrind(i+1, "Mahlen") for i in range(6)]
press = [CoffeePress(i+1, "Pressen") for i in range(5)]
moisting = [PowderMoister(i+1, "Angefeuchten") for i in range(2)]
brewing = [CoffeeBrewing(i+1, "Brühen") for i in range(14)]
toStartPosition = [BrewingUnitToStartPosition(i+1, "Zur Startposition") for i in range(4)]

coffee_types = {
    "1": {"name": "Normal", "grinding": grinding, "press": press, "moisting": moisting, "brewing": brewing, "toStart": toStartPosition},
    "2": {"name": "Espresso", "grinding": grinding, "press": press, "moisting": moisting, "brewing": brewing, "toStart": toStartPosition}
}


Durchlauf von einem Kaffee der runtergedrückt wird

In [3]:
state = MachineState()

while True:
    option = input("\nMaschine einschalten (1)\nKaffee runterlassen (2)\nWassertank füllen (3)\nKaffeesatz leeren (4)\nAusschalten (5)\n> ")

    if option == "1":
        print(">>> Maschine heizt auf...")
        heatingUp = [HeatUp(sec, temp) for sec, temp in enumerate([
            22, 27, 31, 34, 37, 40, 43, 45, 47, 49, 
            50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 
            70, 72, 74, 76, 78, 80, 82, 84, 86, 87, 
            88, 89, 90, 90, 91, 91, 92, 92, 92, 92, 
            92, 93, 93, 93, 93, 93, 93, 93, 93, 93, 
            94, 94, 94, 94, 94
        ])]
        for step in heatingUp:
            step.execute(state)
        print(">>> Maschine ist betriebsbereit!")

    elif option == "2":
        coffee_choice = input("Wähle Kaffee:\n1 = Normal\n2 = Espresso\n> ")
        if coffee_choice in coffee_types:
            recipe = coffee_types[coffee_choice]
            if not state.water_ok:
                print(">>> Nicht genug Wasser! Bitte Wasser nachfüllen.")
                continue
            if not state.grounds_ok:
                print(">>> Kaffeesatzbehälter voll! Bitte leeren.")
                continue

            print(f">>> Starte {recipe['name']}...")

            for step_list in ["grinding", "press", "moisting"]:
                for step in recipe[step_list]:
                    step.execute(state)

            for step in recipe["brewing"]:
                ok = step.execute(state)
                state.cups_since_empty = state.cups_since_empty + 1
                if not ok:
                    break

            for step in recipe["toStart"]:
                step.execute(state)

            print(">>> Kaffee ist fertig!")

        else:
            print("Ungültige Auswahl!")

    elif option == "3":
        state.water_ok = True
        print(">>> Wassertank aufgefüllt.")

    elif option == "4":
        state.grounds_ok = True
        state.cups_since_empty = 0
        print(">>> Kaffeesatzbehälter geleert.")

    elif option == "5":
        break

    else:
        print("Ungültige Eingabe!")

>>> Maschine heizt auf...
Aufheizen | Temp: 22°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 27°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 31°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 34°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 37°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 40°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 43°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 45°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 47°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025
Aufheizen | Temp: 49°C, Wasser: OK, Kaffeesatz: OK, Wasserdurchfluss: 0ml/s, Datum: 22.08.2025


KeyboardInterrupt: 